# **Install Package**

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.6/913.6 kB 19.4 MB/s eta 0:00:00


# **Import Package**

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os
from ultralytics import YOLO
from sklearn.metrics import precision_recall_fscore_support, average_precision_score

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Object Tracking, Deteksi Anomali, dan Klasifikasi**

In [ ]:
import time
import os
import cv2
import numpy as np
import tensorflow as tf

# Load model YOLOv9 untuk deteksi objek
yolo_model = YOLO('/content/drive/MyDrive/Skripsi/Model YOLOV9 New/Result Model 1 (70:30)/weights/best.pt')

# Load model MobileNet untuk klasifikasi
model = tf.keras.models.load_model('/content/drive/MyDrive/Skripsi/Model MobileNetv2/Result Skenario 2 (80:20) Dataset 3/my_model.h5')

# Membuat direktori jika belum ada
os.makedirs('/content/anomali', exist_ok=True)
os.makedirs('/content/anomaliwithboundingbox', exist_ok=True)  # Direktori untuk frame dengan bounding box

# Fungsi untuk mengklasifikasi gambar menggunakan MobileNet
def classify_image(image):
    resized_image = cv2.resize(image, (224, 224))
    normalized_image = resized_image.astype('float32') / 255.0
    input_image = np.expand_dims(normalized_image, axis=0)
    predictions = model.predict(input_image)

    # Menggunakan sigmoid untuk hasil prediksi
    predicted_class = (predictions[0] > 0.5).astype(int)  # Threshold 0.5 untuk prediksi biner
    class_labels = ['accident','no-accident' ]  # Disesuaikan dengan label output Anda

    return class_labels[predicted_class[0]]

# Fungsi untuk menghitung rata-rata bergerak
def moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size) / window_size, mode='valid')

# Fungsi untuk menghitung jarak antara dua titik
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Inisialisasi variabel untuk deteksi anomali
movement_history = {}
speed_history = {}

# Inisialisasi video writer untuk menyimpan hasil tracking
video_input_path = "/content/drive/MyDrive/Skripsi/Data Video/Data Uji/accident/Bubutan.mp4"
video_capture = cv2.VideoCapture(video_input_path)
fps = video_capture.get(cv2.CAP_PROP_FPS)
frame_size = (int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter("/content/tracking_results/tracking_output.mp4", fourcc, fps, frame_size)

# Melakukan tracking objek pada video menggunakan YOLOv9
results = yolo_model.track(source=video_input_path, save=True, stream=True, persist=True, tracker="bytetrack.yaml", show=True)

frame_num = 0

# Variabel untuk menyimpan informasi anomali
anomaly_info = []

# Loop utama untuk memproses setiap frame
for result in results:
    if result.boxes is None or len(result.boxes) == 0:
        frame_num += 1
        continue

    boxes = result.boxes
    annotated_frame = result.orig_img.copy()

    for box in boxes:
        obj_id = box.id.item() if box.id is not None else None
        class_id = int(box.cls) if box.cls is not None else None
        label = yolo_model.names[class_id] if class_id is not None else "Unknown"

        if obj_id is None:
            continue

        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
        center = ((x1 + x2) / 2, (y1 + y2) / 2)

        if obj_id not in movement_history:
            movement_history[obj_id] = []
            speed_history[obj_id] = []

        movement_history[obj_id].append(center)

        if len(movement_history[obj_id]) > 1:
            prev_center = movement_history[obj_id][-2]
            current_speed = calculate_distance(center, prev_center)
            speed_history[obj_id].append(current_speed)

            if len(speed_history[obj_id]) >= 25:
                avg_speed = moving_average(speed_history[obj_id], window_size=25)[-1]

                # Deteksi berhenti mendadak
                if avg_speed > 5 and current_speed < 3:
                    # Simpan informasi anomali
                    anomaly_info.append({
                        "frame_num": frame_num,
                        "label": label,
                        "obj_id": obj_id,
                        "current_speed": current_speed,
                        "avg_speed": avg_speed,
                        "image_no_bbox": result.orig_img.copy(),  # Frame tanpa bounding box
                        "bounding_box": (x1, y1, x2, y2)
                    })

                    # Gambarkan bounding box merah pada frame anomali
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    text = f"Anomaly: Speed={current_speed:.2f}, Avg={avg_speed:.2f}"
                    cv2.putText(annotated_frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                    # Simpan frame dengan bounding box untuk digunakan nanti
                    anomaly_info[-1]["image_with_bbox"] = annotated_frame.copy()

    frame_num += 1

# Pilih top 2 frame dengan anomali
top_anomalies = sorted(anomaly_info, key=lambda x: x['avg_speed'] - x['current_speed'], reverse=True)[:2]

# Simpan frame di dua folder
for idx, anomaly in enumerate(top_anomalies, start=1):
    # Simpan frame tanpa bounding box
    no_bbox_path = f"/content/anomali/top_anomaly_{idx:02d}_frame_{anomaly['frame_num']:04d}.jpg"
    cv2.imwrite(no_bbox_path, anomaly["image_no_bbox"])

    # Simpan frame dengan bounding box
    with_bbox_path = f"/content/anomaliwithboundingbox/top_anomaly_{idx:02d}_frame_{anomaly['frame_num']:04d}_bbox.jpg"
    cv2.imwrite(with_bbox_path, anomaly["image_with_bbox"])

    # Cetak informasi anomali
    print(f"Frame {anomaly['frame_num']} - Object: {anomaly['label']}, current_speed: {anomaly['current_speed']}, avg_speed: {anomaly['avg_speed']}")

# Klasifikasi frame anomali tanpa bounding box
for anomalous_frame in os.listdir('/content/anomali'):
    frame_path = os.path.join('/content/anomali', anomalous_frame)
    frame = cv2.imread(frame_path)
    accident_class = classify_image(frame)
    print(f"Frame {anomalous_frame}: Detected {accident_class}")

print("Proses selesai.")


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.5s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()


video 1/1 (frame 1/323) /content/drive/MyDrive/Skripsi/Data Video/Data Uji/accident/Bubutan.mp4: 384x640 1 car, 4 motorcycles, 53.9ms
video 1/1 (frame 2/323) /content/drive/MyDrive/Skripsi/Data Video/Data Uji/accident/Bubutan.mp4: 384x640 1 car, 4 motorcycles, 35.2ms
video 1/1 (frame 3/323) /content/drive/MyDrive/Skripsi/Data Video/Data Uji/accident/Bubutan.mp4: 384x640 1 car, 3 motorcycles, 31.4ms
video 1/1 (frame 4/323) /content/drive/MyDrive/Skripsi/Data Video/Data Uji/accident/Bubutan.mp4: 384x640 1 car, 3 motorcycles, 31.4ms
video 1/1 (frame 5/323) /content/drive/MyDrive/Skripsi/Data Video/



---

---





In [ ]:
import shutil

# Tentukan path folder yang ingin dihapus
folder_path = '/content/runs'  # Ganti dengan path folder yang ingin Anda hapus

# Hapus folder dan seluruh isinya
shutil.rmtree(folder_path)

print(f"Folder {folder_path} berhasil dihapus beserta isinya.")


Folder /content/runs berhasil dihapus beserta isinya.


In [ ]:
import shutil

# Tentukan path folder yang ingin dihapus
folder_path = '/content/anomali'  # Ganti dengan path folder yang ingin Anda hapus

# Hapus folder dan seluruh isinya
shutil.rmtree(folder_path)

print(f"Folder {folder_path} berhasil dihapus beserta isinya.")


Folder /content/anomali berhasil dihapus beserta isinya.


In [ ]:
import shutil

# Tentukan path folder yang ingin dihapus
folder_path = '/content/anomaliwithboundingbox'  # Ganti dengan path folder yang ingin Anda hapus

# Hapus folder dan seluruh isinya
shutil.rmtree(folder_path)

print(f"Folder {folder_path} berhasil dihapus beserta isinya.")


Folder /content/anomaliwithboundingbox berhasil dihapus beserta isinya.
